In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear
from torch_geometric.data import Data

In [4]:
hidden_dim = 64
output_dim = 9

In [5]:
adj_matrix = pd.read_csv("data/science.add9330_data_s1/Supplementary-Data-S1/all-all_connectivity_matrix.csv", index_col=0)
num_nodes = adj_matrix.shape[0]

In [6]:
edge_index = torch.tensor(np.vstack(np.where(adj_matrix > 0)), dtype=torch.long)  # Edges are where adj_matrix > 0
edge_attr = torch.tensor(adj_matrix[adj_matrix > 0], dtype=torch.float)  # Edge attributes are the non-zero values of adj_matrix

In [ ]:

input_node_size = 0

In [9]:
edge_attr

tensor([9., 1., 2.,  ..., 3., 1., 1.])

In [10]:
feature_dim = 10  # Choose an appropriate value for your case
node_features = torch.randn(num_nodes, feature_dim)

In [11]:
# Network
# Define the Convolutional Network
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = Conv2d(1, 32, kernel_size=5)
        self.conv2 = Conv2d(32, 64, kernel_size=5)
        self.fc1 = Linear(64 * 4 * 4, input_node_size)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        return x

# Define the Graph Convolutional Network
class GCN(torch.nn.Module):
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(feature_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

# Define the Combined Model
class CombinedModel(torch.nn.Module):
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(CombinedModel, self).__init__()
        self.convnet = ConvNet()
        self.gcn = GCN(feature_dim, hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.convnet(x)
        # Reshape the output from the ConvNet to match the input shape of the GCN
        data.x = x.view(-1, input_node_size)
        x = self.gcn(data)
        return x

In [24]:
# Preprocess the image
# Suppose you have an image of size n*m and a corresponding adjacency matrix
a = 100 # Number of training images
n, m = 128, 128 # Pixel dimensions of each image
num_nodes = n * m
feature_dim = 1  # Grayscale image, for RGB images use feature_dim = 3

# Let's assume image is your n*m numpy array image
image = np.random.rand(a, n, m)
# image = image.reshape(-1, 1)  # Reshape to a num_nodes*1 array

# Convert numpy array to tensor and assign it to data.x
# x = torch.from_numpy(image).float()

x = torch.randn(100, 1, 128, 128)
# Create a PyG Data object
data = Data(x=x, edge_index=edge_index)

In [25]:
x.shape

torch.Size([100, 1, 128, 128])

In [26]:
# Initialize the model
model = CombinedModel(feature_dim=500, hidden_dim=16, output_dim=10)

# Define a loss function and an optimizer
loss_fn = torch.nn.CrossEntropyLoss()  # or another suitable loss function for your problem
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # or another optimizer

In [27]:
# Training loop
for epoch in range(5):  # or however many epochs you want to train for
    model.train()
    optimizer.zero_grad()

    # Forward pass
    out = model(data)

    # Compute the loss, gradients, and update model parameters
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x53824 and 1024x500)

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

0